In [ ]:
%pip install spacy


In [12]:
%pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 3.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install ipyfilechooser

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
conda list spacy

In [ ]:
pip list

In [ ]:
%pip install pytesseract opencv-python-headless pillow numpy

In [ ]:
%pip install pandas

In [ ]:
%pip install ipywidgets

In [ ]:
%pip install opencv-python-headless

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
from tkinter import Tk, filedialog
import os

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# Function to preprocess the image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

# Function to extract text using Tesseract OCR
def extract_text(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

# Function to process text using NLP and extract relevant information
def process_text(text, keywords=[]):
    doc = nlp(text)
    extracted_info = []

    for ent in doc.ents:
        if any(keyword.lower() in ent.text.lower() for keyword in keywords):
            extracted_info.append({"Text": ent.text, "Label": ent.label_})

    return extracted_info

# Function to select image using a button in Jupyter Notebook
def select_image(_):
    root = Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select an Image", filetypes=[("JPEG Files", "*.jpg"), ("PNG Files", "*.png"), ("JPEG Files", "*.jpeg")])

    if file_path and os.path.exists(file_path):
        process_image_jupyter(file_path)
    else:
        with output_widget:
            output_widget.clear_output()
            print("Error: No valid file selected.")

# Function to process selected image within Jupyter environment
def process_image_jupyter(image_path):
    try:
        text = extract_text(image_path)
        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the image.")
                return


            print("Full Extracted Text:", text)

            important_info = process_text(text, ["date", "amount", "name", "address", "Name:", "Customer Name", "Recipient", "Amount:", "Date:", "Address:", "ID:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

button = widgets.Button(description="Select Image")
button.on_click(select_image)

display(button, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

In [ ]:
%pip install pdfplumber

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def process_text(text, keywords=[]):
    extracted_info = []

    for keyword in keywords:
        if keyword in ["Name:", "Customer Name", "Recipient", "Name"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Address:", "Address"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s,-]+(?:\n[\w\s,-]+)*)")
        elif keyword in ["Date:", "Date"]:
            pattern = re.compile(rf"{keyword}\s*([\d/]+)")
        elif keyword in ["Amount:", "Amount"]:
            pattern = re.compile(rf"{keyword}\s*([\d]+)")
        elif keyword in ["ID:", "ID"]:
            pattern = re.compile(rf"{keyword}\s*([\d]+)")
        else:
            continue

        for match in pattern.finditer(text):  # Use finditer to find all matches
            extracted_text = match.group(1).strip()
            # Remove any text after "\n"
            extracted_text = extracted_text.split("\n")[0].strip()
            extracted_info.append({"Text": extracted_text, "Label": keyword.replace(":", "")})

    return extracted_info

def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Address:", "Date:", "Amount:", "ID:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector 

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def process_text(text, keywords=[]):
    extracted_info = []

    for keyword in keywords:
        if keyword in ["Name:", "Customer Name", "Recipient", "Name"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Address:", "Address"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s,-]+(?:\n[\w\s,-]+)*)")
        elif keyword in ["Date:", "Date"]:
            pattern = re.compile(rf"{keyword}\s*([\d/]+)")
        elif keyword in ["Amount:", "Amount"]:
            pattern = re.compile(rf"{keyword}\s*([\d]+)")
        elif keyword in ["ID:", "ID"]:
            pattern = re.compile(rf"{keyword}\s*([\d]+)")
        else:
            continue

        for match in pattern.finditer(text):  # Use finditer to find all matches
            extracted_text = match.group(1).strip()
            # Remove any text after "\n"
            extracted_text = extracted_text.split("\n")[0].strip()
            extracted_info.append({"Text": extracted_text, "Label": keyword.replace(":", "")})

    return extracted_info

def save_to_database(data):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        for item in data:
            query = "INSERT INTO extracted_data (text, label) VALUES (%s, %s)"
            values = (item["Text"], item["Label"])
            cursor.execute(query, values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")

def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Address:", "Date:", "Amount:", "ID:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)
                save_to_database(important_info)  # Save to database here
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

The current implementation will indeed lead to a problem when using the stored data in the future. Each time you process a new document and extract information, it is being appended to the same extracted_data table. This means you will have a growing list of all the extracted information, without any clear distinction between the data from different documents.

Here's a breakdown of why this is a problem and how it could cause issues:

Data Mixing: All the extracted "Name," "Address," "Date," "Amount," and "ID" will be in a single table, making it difficult to associate specific details with their original document.
Difficulty in Analysis: If you want to analyze the data, for example, to see the total number of claims for a specific date range or identify claims from a particular customer, it will be very challenging to filter and group the data correctly.
Redundancy: If you process multiple documents from the same claim or the same person, you might end up with redundant entries in the database.
Query Complexity: Writing queries to retrieve specific information will become more complex and prone to errors.

Hence, the following changes:

In [2]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def process_text(text, keywords=[]):
    extracted_info = []

    for keyword in keywords:
        if keyword in ["Name:", "Customer Name", "Recipient", "Name"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Address:", "Address"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s,-]+(?:\n[\w\s,-]+)*)")
        elif keyword in ["Date:", "Date"]:
            pattern = re.compile(rf"{keyword}\s*([\d/]+)")
        elif keyword in ["Amount:", "Amount"]:
            pattern = re.compile(rf"{keyword}\s*([\d]+)")
        elif keyword in ["ID:", "ID"]:
            pattern = re.compile(rf"{keyword}\s*([\d]+)")
        else:
            continue

        for match in pattern.finditer(text):  # Use finditer to find all matches
            extracted_text = match.group(1).strip()
            # Remove any text after "\n"
            extracted_text = extracted_text.split("\n")[0].strip()
            extracted_info.append({"Text": extracted_text, "Label": keyword.replace(":", "")})

    return extracted_info

def save_to_database(data, document_id):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        for item in data:
            query = "INSERT INTO extracted_data (document_id, text, label) VALUES (%s, %s, %s)"
            values = (document_id, item["Text"], item["Label"])
            cursor.execute(query, values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")

def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Address:", "Date:", "Amount:", "ID:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                # Save document information to the 'documents' table
                cnx = mysql.connector.connect(**DB_CONFIG)
                cursor = cnx.cursor()
                insert_doc_query = "INSERT INTO documents (file_path) VALUES (%s)"
                doc_values = (file_path,)
                cursor.execute(insert_doc_query, doc_values)
                cnx.commit()
                document_id = cursor.lastrowid  # Get the auto-generated ID
                cursor.close()
                cnx.close()

                # Save extracted information with the document_id
                save_to_database(important_info, document_id)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

Now I want to make some changes. The changes I would like are is that I want to extract more key information that must include:
Name, Father's Name, Adhaar Card, Gender, Blood Group, Address, Hospital Name, Insurance id,Appointment time, phone number,amount, disease_name,diesease_details, Medicines needed including injections,oral,etc and also Bedtype details including bedtype and ventilation: yes/no as well as if other charges. Before the primary key was document_id was the primary key, now I want to make the insurance_id as the primary key.
To not confuse with the previous tables and its already stored data I would suggest to create new tables. Do the proper changes.

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

In [18]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def process_text(text, keywords=[]):
    extracted_info = []

    for keyword in keywords:
        if keyword in ["Name:", "Customer Name", "Recipient", "Name", "Patient Name"]:
            pattern = re.compile(rf"{keyword}:\s*([\w\s]+(?:[\n\w\s]+)*)", re.IGNORECASE)
        elif keyword in ["Father's Name:", "Father Name:"]:
            pattern = re.compile(rf"(?<!\w){keyword}:\s*([\w\s]+(?:[\n\w\s]+)*)", re.IGNORECASE)
        elif keyword in ["Aadhar Card:", "Aadhaar:", "Aadhar No:", "Aadhar"]:
            pattern = re.compile(rf"{keyword}:\s*([\d\s-]+)", re.IGNORECASE)
        elif keyword in ["Gender:", "Sex:"]:
            pattern = re.compile(rf"{keyword}:\s*([\w]+)", re.IGNORECASE)
        elif keyword in ["Blood Group:", "Blood:"]:
            pattern = re.compile(rf"{keyword}:\s*([\w+-]+)", re.IGNORECASE)
        elif keyword in ["Address:", "Address"]:
            pattern = re.compile(rf"{keyword}:\s*([\w\s,-]+(?:[\n\w\s,-]+)*)", re.IGNORECASE)
        elif keyword in ["Hospital Name:", "Hospital"]:
            pattern = re.compile(rf"(?<!\w){keyword}:\s*([\w\s]+(?:[\n\w\s]+)*)", re.IGNORECASE)
        elif keyword in ["Insurance ID:", "InsuranceID:", "Insurance Id", "Insurance"]:
            pattern = re.compile(rf"{keyword}:\s*([\w\d-]+)", re.IGNORECASE)
        elif keyword in ["Appointment Time:", "Appointment", "Time:"]:
            pattern = re.compile(rf"{keyword}:\s*([\d:]+(?:\s*[AP]M)?)", re.IGNORECASE)
        elif keyword in ["Phone:", "Phone Number:", "Contact:", "Mobile:"]:
            pattern = re.compile(rf"{keyword}:\s*([\d\s-]+)", re.IGNORECASE)
        elif keyword in ["Amount:", "Amount", "Total:"]:
            pattern = re.compile(rf"{keyword}:\s*([\d\.]+)", re.IGNORECASE)
        elif keyword in ["Disease Name:", "Disease:", "Diagnosis:"]:
            pattern = re.compile(rf"(?<!\w){keyword}:\s*([\w\s]+(?:[\n\w\s]+)*)", re.IGNORECASE)
        elif keyword in ["Disease Details:", "Details:"]:
            pattern = re.compile(rf"{keyword}:\s*([\w\s.,-]+(?:[\n\w\s.,-]+)*)", re.IGNORECASE)
        elif keyword in ["Medicines:", "Medication:", "Medicine:", "Injections:", "Oral:"]:
            pattern = re.compile(rf"{keyword}:\s*([\w\s.,-]+(?:[\n\w\s.,-]+)*)", re.IGNORECASE)
        elif keyword in ["Bed Type:", "Bed:"]:
            pattern = re.compile(rf"{keyword}:\s*([\w\s-]+)", re.IGNORECASE)
        elif keyword in ["Ventilation:", "Ventilator:"]:
            pattern = re.compile(rf"{keyword}:\s*(Yes|No)", re.IGNORECASE)
        elif keyword in ["Other Charges:", "Charges:"]:
            pattern = re.compile(rf"{keyword}:\s*([\d\.]+)", re.IGNORECASE)

        else:
            continue

        for match in pattern.finditer(text):
            if match:
                print(match.group(0)) #adding debugging line
                extracted_text = match.group(1).strip()
                extracted_text = extracted_text.split("\n")[0].strip()
                extracted_info.append({"Text": extracted_text, "Label": keyword.replace(":", "")})
    print(extracted_info) #Debugging line
    return extracted_info

def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        # Insert document information into the 'patient_documents' table
        insert_doc_query = "INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)"
        doc_values = (insurance_id, file_path)
        cursor.execute(insert_doc_query, doc_values)

        # Insert patient details into the 'patient_details' table
        insert_patient_query = "INSERT INTO patient_details (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        name = next((item["Text"] for item in data if item["Label"] == "Name"), None)
        father_name = next((item["Text"] for item in data if item["Label"] == "Father's Name"), None)
        aadhar_card = next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None)
        gender = next((item["Text"] for item in data if item["Label"] == "Gender"), None)
        blood_group = next((item["Text"] for item in data if item["Label"] == "Blood Group"), None)
        address = next((item["Text"] for item in data if item["Label"] == "Address"), None)
        hospital_name = next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None)
        appointment_time = next((item["Text"] for item in data if item["Label"] == "Appointment Time"), None)
        phone_number = next((item["Text"] for item in data if item["Label"] == "Phone Number"), None)
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        disease_name = next((item["Text"] for item in data if item["Label"] == "Disease Name"), None)
        disease_details = next((item["Text"] for item in data if item["Label"] == "Disease Details"), None)
        medicines = next((item["Text"] for item in data if item["Label"] == "Medicines"), None)
        bed_type = next((item["Text"] for item in data if item["Label"] == "Bed Type"), None)
        ventilation = next((item["Text"] for item in data if item["Label"] == "Ventilation"), None)
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)

        patient_values = (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges)
        print(patient_values) #Debugging line
        cursor.execute(insert_patient_query, patient_values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")

def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

In [14]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def process_text(text, keywords=[]):
    print(text) # Debugging line
    extracted_info = []

    for keyword in keywords:
        pattern = re.compile(rf"{keyword}:\s*(.*)", re.IGNORECASE) #Simplified regex for test

        for match in pattern.finditer(text):
            if match:
                print(match.group(0)) #adding debugging line
                extracted_text = match.group(1).strip()
                extracted_text = extracted_text.split("\n")[0].strip()
                extracted_info.append({"Text": extracted_text, "Label": keyword.replace(":", "")})
    print(extracted_info) #Debugging line
    return extracted_info

def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        # Insert document information into the 'patient_documents' table
        insert_doc_query = "INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)"
        doc_values = (insurance_id, file_path)
        cursor.execute(insert_doc_query, doc_values)

        # Insert patient details into the 'patient_details' table
        insert_patient_query = "INSERT INTO patient_details (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        name = next((item["Text"] for item in data if item["Label"] == "Name"), None)
        father_name = next((item["Text"] for item in data if item["Label"] == "Father's Name"), None)
        aadhar_card = next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None)
        gender = next((item["Text"] for item in data if item["Label"] == "Gender"), None)
        blood_group = next((item["Text"] for item in data if item["Label"] == "Blood Group"), None)
        address = next((item["Text"] for item in data if item["Label"] == "Address"), None)
        hospital_name = next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None)
        appointment_time = next((item["Text"] for item in data if item["Label"] == "Appointment Time"), None)
        phone_number = next((item["Text"] for item in data if item["Label"] == "Phone Number"), None)
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        disease_name = next((item["Text"] for item in data if item["Label"] == "Disease Name"), None)
        disease_details = next((item["Text"] for item in data if item["Label"] == "Disease Details"), None)
        medicines = next((item["Text"] for item in data if item["Label"] == "Medicines"), None)
        bed_type = next((item["Text"] for item in data if item["Label"] == "Bed Type"), None)
        ventilation = next((item["Text"] for item in data if item["Label"] == "Ventilation"), None)
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)

        patient_values = (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges)
        print(patient_values) #Debugging line
        cursor.execute(insert_patient_query, patient_values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")

def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

In [17]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def process_text(text, keywords=[]):
    print("Text before regex:")
    print(text)  # Debugging line

    extracted_info = []

    for char in text:
        print(ord(char), char)

    text = re.sub(r'[^\w\s:]', '', text)

    print("Text after non alpha removal:")
    print(text)

    for keyword in keywords:
        pattern = re.compile(rf"{keyword}:\s*(.*)", re.IGNORECASE)

        for match in pattern.finditer(text):
            if match:
                print(match.group(0))
                extracted_text = match.group(1).strip()
                extracted_text = extracted_text.split("\n")[0].strip()
                extracted_info.append({"Text": extracted_text, "Label": keyword.replace(":", "")})
    print(extracted_info)
    return extracted_info

def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        # Insert document information into the 'patient_documents' table
        insert_doc_query = "INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)"
        doc_values = (insurance_id, file_path)
        cursor.execute(insert_doc_query, doc_values)

        # Insert patient details into the 'patient_details' table
        insert_patient_query = "INSERT INTO patient_details (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        name = next((item["Text"] for item in data if item["Label"] == "Name"), None)
        father_name = next((item["Text"] for item in data if item["Label"] == "Father's Name"), None)
        aadhar_card = next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None)
        gender = next((item["Text"] for item in data if item["Label"] == "Gender"), None)
        blood_group = next((item["Text"] for item in data if item["Label"] == "Blood Group"), None)
        address = next((item["Text"] for item in data if item["Label"] == "Address"), None)
        hospital_name = next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None)
        appointment_time = next((item["Text"] for item in data if item["Label"] == "Appointment Time"), None)
        phone_number = next((item["Text"] for item in data if item["Label"] == "Phone Number"), None)
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        disease_name = next((item["Text"] for item in data if item["Label"] == "Disease Name"), None)
        disease_details = next((item["Text"] for item in data if item["Label"] == "Disease Details"), None)
        medicines = next((item["Text"] for item in data if item["Label"] == "Medicines"), None)
        bed_type = next((item["Text"] for item in data if item["Label"] == "Bed Type"), None)
        ventilation = next((item["Text"] for item in data if item["Label"] == "Ventilation"), None)
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)

        patient_values = (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges)
        print(patient_values) #Debugging line
        cursor.execute(insert_patient_query, patient_values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")

def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

Atleast Working Code:

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def process_text(text, keywords=[]):
    extracted_info = []

    for keyword in keywords:
        if keyword in ["Name:", "Customer Name", "Recipient", "Name", "Patient Name"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Father's Name:", "Father Name:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Aadhar Card:", "Aadhaar:", "Aadhar No:", "Aadhar"]:
            pattern = re.compile(rf"{keyword}\s*([\d\s-]+)")
        elif keyword in ["Gender:", "Sex:"]:
            pattern = re.compile(rf"{keyword}\s*([\w]+)")
        elif keyword in ["Blood Group:", "Blood:"]:
            pattern = re.compile(rf"{keyword}\s*([\w+-]+)")
        elif keyword in ["Address:", "Address"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s,-]+(?:\n[\w\s,-]+)*)")
        elif keyword in ["Hospital Name:", "Hospital"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Insurance ID:", "InsuranceID:", "Insurance Id", "Insurance"]:
            pattern = re.compile(rf"{keyword}\s*([\w\d-]+)")
        elif keyword in ["Appointment Time:", "Appointment", "Time:"]:
            pattern = re.compile(rf"{keyword}\s*([\d:]+(?:\s*[AP]M)?)")
        elif keyword in ["Phone:", "Phone Number:", "Contact:", "Mobile:"]:
            pattern = re.compile(rf"{keyword}\s*([\d\s-]+)")
        elif keyword in ["Amount:", "Amount", "Total:"]:
            pattern = re.compile(rf"{keyword}\s*([\d\.]+)")
        elif keyword in ["Disease Name:", "Disease:", "Diagnosis:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Disease Details:", "Details:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s.,-]+(?:\n[\w\s.,-]+)*)")
        elif keyword in ["Medicines:", "Medication:", "Medicine:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s.,-]+(?:\n[\w\s.,-]+)*)")
        elif keyword in ["Bed Type:", "Bed:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+)")
        elif keyword in ["Ventilation:", "Ventilator:"]:
            pattern = re.compile(rf"{keyword}\s*(Yes|No)")
        elif keyword in ["Other Charges:", "Charges:"]:
            pattern = re.compile(rf"{keyword}\s*([\d\.]+)")

        else:
            continue

        for match in pattern.finditer(text):  # Use finditer to find all matches
            extracted_text = match.group(1).strip()
            # Remove any text after "\n"
            extracted_text = extracted_text.split("\n")[0].strip()
            extracted_info.append({"Text": extracted_text, "Label": keyword.replace(":", "")})

    return extracted_info

def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        # Insert document information into the 'patient_documents' table
        insert_doc_query = "INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)"
        doc_values = (insurance_id, file_path)
        cursor.execute(insert_doc_query, doc_values)

        # Insert patient details into the 'patient_details' table
        insert_patient_query = "INSERT INTO patient_details (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        name = next((item["Text"] for item in data if item["Label"] == "Name"), None)
        father_name = next((item["Text"] for item in data if item["Label"] == "Father's Name"), None)
        aadhar_card = next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None)
        gender = next((item["Text"] for item in data if item["Label"] == "Gender"), None)
        blood_group = next((item["Text"] for item in data if item["Label"] == "Blood Group"), None)
        address = next((item["Text"] for item in data if item["Label"] == "Address"), None)
        hospital_name = next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None)
        appointment_time = next((item["Text"] for item in data if item["Label"] == "Appointment Time"), None)
        phone_number = next((item["Text"] for item in data if item["Label"] == "Phone Number"), None)
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        disease_name = next((item["Text"] for item in data if item["Label"] == "Disease Name"), None)
        disease_details = next((item["Text"] for item in data if item["Label"] == "Disease Details"), None)
        medicines = next((item["Text"] for item in data if item["Label"] == "Medicines"), None)
        bed_type = next((item["Text"] for item in data if item["Label"] == "Bed Type"), None)
        ventilation = next((item["Text"] for item in data if item["Label"] == "Ventilation"), None)
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)

        patient_values = (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges)
        cursor.execute(insert_patient_query, patient_values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")

def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

Archisman's

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def process_text(text, keywords=[]):
    extracted_info = []

    for keyword in keywords:
        if keyword in ["Name:", "Customer Name", "Recipient", "Name", "Patient Name"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Father's Name:", "Father Name:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Aadhar Card:", "Aadhaar:", "Aadhar No:", "Aadhar"]:
            pattern = re.compile(rf"{keyword}\s*([\d\s-]+)")
        elif keyword in ["Gender:", "Sex:"]:
            pattern = re.compile(rf"{keyword}\s*([\w]+)")
        elif keyword in ["Blood Group:", "Blood:"]:
            pattern = re.compile(rf"{keyword}\s*([\w+-]+)")
        elif keyword in ["Address:", "Address"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s,-]+(?:\n[\w\s,-]+)*)")
        elif keyword in ["Hospital Name:", "Hospital"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Insurance ID:", "InsuranceID:", "Insurance Id", "Insurance"]:
            pattern = re.compile(rf"{keyword}\s*([\w\d-]+)")
        elif keyword in ["Appointment Time:", "Appointment", "Time:"]:
            pattern = re.compile(rf"{keyword}\s*([\d:]+(?:\s*[AP]M)?)")
        elif keyword in ["Phone:", "Phone Number:", "Contact:", "Mobile:"]:
            pattern = re.compile(rf"{keyword}\s*([\d\s-]+)")
        elif keyword in ["Amount:", "Amount", "Total:"]:
            pattern = re.compile(rf"{keyword}\s*([\d\.]+)")
        elif keyword in ["Disease Name:", "Disease:", "Diagnosis:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Disease Details:", "Details:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s.,-]+(?:\n[\w\s.,-]+)*)")
        elif keyword in ["Medicines:", "Medication:", "Medicine:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s.,-]+(?:\n[\w\s.,-]+)*)")
        elif keyword in ["Bed Type:", "Bed:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+)")
        elif keyword in ["Ventilation:", "Ventilator:"]:
            pattern = re.compile(rf"{keyword}\s*(Yes|No)")
        elif keyword in ["Other Charges:", "Charges:"]:
            pattern = re.compile(rf"{keyword}\s*([\d\.]+)")

        else:
            continue

        for match in pattern.finditer(text):  # Use finditer to find all matches
            extracted_text = match.group(1).strip()
            # Remove any text after "\n"
            extracted_text = extracted_text.split("\n")[0].strip()
            extracted_info.append({"Text": extracted_text, "Label": keyword.replace(":", "")})

    return extracted_info

def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        # Insert document information into the 'patient_documents' table
        insert_doc_query = "INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)"
        doc_values = (insurance_id, file_path)
        cursor.execute(insert_doc_query, doc_values)

        # Insert patient details into the 'patient_details' table
        insert_patient_query = "INSERT INTO patient_details (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        name = next((item["Text"] for item in data if item["Label"] == "Name"), None)
        father_name = next((item["Text"] for item in data if item["Label"] == "Father's Name"), None)
        aadhar_card = next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None)
        gender = next((item["Text"] for item in data if item["Label"] == "Gender"), None)
        blood_group = next((item["Text"] for item in data if item["Label"] == "Blood Group"), None)
        address = next((item["Text"] for item in data if item["Label"] == "Address"), None)
        hospital_name = next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None)
        appointment_time = next((item["Text"] for item in data if item["Label"] == "Appointment Time"), None)
        phone_number = next((item["Text"] for item in data if item["Label"] == "Phone Number"), None)
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        disease_name = next((item["Text"] for item in data if item["Label"] == "Disease Name"), None)
        disease_details = next((item["Text"] for item in data if item["Label"] == "Disease Details"), None)
        medicines = next((item["Text"] for item in data if item["Label"] == "Medicines"), None)
        bed_type = next((item["Text"] for item in data if item["Label"] == "Bed Type"), None)
        ventilation = next((item["Text"] for item in data if item["Label"] == "Ventilation"), None)
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)

        patient_values = (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges)
        cursor.execute(insert_patient_query, patient_values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")

def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

Archisman S2

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def process_text(text, keywords=[]):
    extracted_info = []
    for keyword in keywords:
        pattern = re.compile(rf"{keyword}\s*([\w\s.,-]+(?:\n[\w\s.,-]+)*)", re.IGNORECASE)
        for match in pattern.finditer(text):
            extracted_text = match.group(1).strip()
            extracted_text = re.sub(r'\n+', ' ', extracted_text)  # Remove extra newlines
            if extracted_text:
                extracted_info.append({"Text": extracted_text, "Label": keyword.replace(":", "")})
    return extracted_info

def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()
        cursor.execute("INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)", (insurance_id, file_path))

        insert_patient_query = """
        INSERT INTO patient_details (insurance_id, name, father_name, aadhar_card, gender, blood_group, address,
        hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type,
        ventilation, other_charges)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        def get_value(label):
            return next((item["Text"] for item in data if item["Label"] == label), None) or None  # Convert "" to None

        patient_values = (
            insurance_id, get_value("Name"), get_value("Father's Name"), get_value("Aadhar Card"),
            get_value("Gender"), get_value("Blood Group"), get_value("Address"), get_value("Hospital Name"),
            get_value("Appointment Time"), get_value("Phone Number"), get_value("Amount"),
            get_value("Disease Name"), get_value("Disease Details"), get_value("Medicines"),
            get_value("Bed Type"), get_value("Ventilation"), get_value("Other Charges")
        )

        cursor.execute(insert_patient_query, patient_values)
        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")

def process_file(file_path):
    try:
        text = extract_text_from_image(file_path) if file_path.lower().endswith(('.png', '.jpg', '.jpeg')) else extract_text_from_pdf(file_path)
        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)
            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)
                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return
                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            print(f"An error occurred: {e}")

file_chooser = FileChooser()
file_chooser.register_callback(lambda chooser: process_file(chooser.selected) if chooser.selected else None)
display(file_chooser, output_widget)
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

Unsuccessful

Resuming Work from previous atleast working Code:

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def process_text(text, keywords=[]):
    extracted_info = []

    # Improve Aadhaar detection - try multiple patterns and formats
    aadhaar_pattern = re.compile(r'(?:Aadhar|Aadhaar|AADHAAR|आधार)(?:\s*(?:Card|Number|No|ID|#|:))?\s*((?:\d[\d\s-]*){12})')
    aadhaar_matches = aadhaar_pattern.findall(text)
    if aadhaar_matches:
        # Clean up the Aadhaar number (remove spaces, dashes)
        aadhaar = re.sub(r'[^\d]', '', aadhaar_matches[0])
        # Format as XXXX-XXXX-XXXX if it's 12 digits
        if len(aadhaar) == 12:
            formatted_aadhaar = f"{aadhaar[:4]}-{aadhaar[4:8]}-{aadhaar[8:]}"
            extracted_info.append({"Text": formatted_aadhaar, "Label": "Aadhar Card"})

    # Improve disease details extraction - handle multi-line content better
    disease_details_pattern = re.compile(r'(?:Disease Details|Details|Diagnosis Details|Clinical Details|Medical Details|Condition Details)[:;]\s*([\w\s,.;:()\-/\n]+)(?=\n\n|\n[A-Z]|$)', re.IGNORECASE)
    disease_details_matches = disease_details_pattern.search(text)
    if disease_details_matches:
        details = disease_details_matches.group(1).strip()
        # Clean up the details
        details = re.sub(r'\n+', ' ', details)
        details = re.sub(r'\s+', ' ', details)
        extracted_info.append({"Text": details, "Label": "Disease Details"})

    # Improve medicines extraction
    medicines_pattern = re.compile(r'(?:Medicines|Medications|Drugs|Prescriptions|Medicine List|Prescribed Medicines)[:;]\s*([\w\s,.;:()\-/\n]+)(?=\n\n|\n[A-Z]|$)', re.IGNORECASE)
    medicines_matches = medicines_pattern.search(text)
    if medicines_matches:
        medicines = medicines_matches.group(1).strip()
        # Clean up the medicines list
        medicines = re.sub(r'\n+', ' ', medicines)
        medicines = re.sub(r'\s+', ' ', medicines)
        extracted_info.append({"Text": medicines, "Label": "Medicines"})

    # Process other keywords as before
    for keyword in keywords:
        # Skip the fields we've already handled with specialized patterns
        if any(term in keyword.lower() for term in ["aadhar", "aadhaar", "disease details", "medicines", "medication"]):
            continue
            
        if keyword in ["Name:", "Customer Name", "Recipient", "Name", "Patient Name"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Father's Name:", "Father Name:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Gender:", "Sex:"]:
            pattern = re.compile(rf"{keyword}\s*([\w]+)")
        elif keyword in ["Blood Group:", "Blood:"]:
            pattern = re.compile(rf"{keyword}\s*([\w+-]+)")
        elif keyword in ["Address:", "Address"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s,-]+(?:\n[\w\s,-]+)*)")
        elif keyword in ["Hospital Name:", "Hospital"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Insurance ID:", "InsuranceID:", "Insurance Id", "Insurance"]:
            pattern = re.compile(rf"{keyword}\s*([\w\d-]+)")
        elif keyword in ["Appointment Time:", "Appointment", "Time:"]:
            pattern = re.compile(rf"{keyword}\s*([\d:]+(?:\s*[AP]M)?)")
        elif keyword in ["Phone:", "Phone Number:", "Contact:", "Mobile:"]:
            pattern = re.compile(rf"{keyword}\s*([\d\s-]+)")
        elif keyword in ["Amount:", "Amount", "Total:"]:
            pattern = re.compile(rf"{keyword}\s*([\d\.]+)")
        elif keyword in ["Disease Name:", "Disease:", "Diagnosis:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Bed Type:", "Bed:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+)")
        elif keyword in ["Ventilation:", "Ventilator:"]:
            pattern = re.compile(rf"{keyword}\s*(Yes|No)")
        elif keyword in ["Other Charges:", "Charges:"]:
            pattern = re.compile(rf"{keyword}\s*([\d\.]+)")
        else:
            continue

        for match in pattern.finditer(text):
            extracted_text = match.group(1).strip()
            # Improved handling of multi-line text - take first line only for certain fields
            if keyword in ["Name:", "Father's Name:", "Gender:", "Blood Group:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Bed Type:", "Ventilation:", "Other Charges:"]:
                extracted_text = extracted_text.split("\n")[0].strip()
            extracted_info.append({"Text": extracted_text, "Label": keyword.replace(":", "")})

    return extracted_info

def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        # Insert document information into the 'patient_documents' table
        insert_doc_query = "INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)"
        doc_values = (insurance_id, file_path)
        cursor.execute(insert_doc_query, doc_values)

        # Insert patient details into the 'patient_details' table
        insert_patient_query = "INSERT INTO patient_details (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        name = next((item["Text"] for item in data if item["Label"] == "Name"), None)
        father_name = next((item["Text"] for item in data if item["Label"] == "Father's Name"), None)
        aadhar_card = next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None)
        gender = next((item["Text"] for item in data if item["Label"] == "Gender"), None)
        blood_group = next((item["Text"] for item in data if item["Label"] == "Blood Group"), None)
        address = next((item["Text"] for item in data if item["Label"] == "Address"), None)
        hospital_name = next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None)
        appointment_time = next((item["Text"] for item in data if item["Label"] == "Appointment Time"), None)
        phone_number = next((item["Text"] for item in data if item["Label"] == "Phone Number"), None)
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        disease_name = next((item["Text"] for item in data if item["Label"] == "Disease Name"), None)
        disease_details = next((item["Text"] for item in data if item["Label"] == "Disease Details"), None)
        medicines = next((item["Text"] for item in data if item["Label"] == "Medicines"), None)
        bed_type = next((item["Text"] for item in data if item["Label"] == "Bed Type"), None)
        ventilation = next((item["Text"] for item in data if item["Label"] == "Ventilation"), None)
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)

        patient_values = (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges)
        cursor.execute(insert_patient_query, patient_values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")

def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

The above code is happening to save the disease details in the database.

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def extract_aadhaar_number(text):
    """
    Specialized function to extract Aadhaar numbers using multiple approaches
    """
    # Clean the text first - this can help with OCR errors
    cleaned_text = re.sub(r'\s+', ' ', text)
    
    # Method 1: Look for common Aadhaar number patterns (12 digits with or without separators)
    # This catches raw numbers that look like Aadhaar numbers
    aadhaar_patterns = [
        # Pattern with no separators - 12 consecutive digits
        r'(?<!\d)(\d{12})(?!\d)',
        # Pattern with space separators
        r'(\d{4}\s+\d{4}\s+\d{4})',
        # Pattern with dash separators
        r'(\d{4}-\d{4}-\d{4})',
        # Pattern with dot separators
        r'(\d{4}\.\d{4}\.\d{4})'
    ]
    
    for pattern in aadhaar_patterns:
        matches = re.findall(pattern, cleaned_text)
        if matches:
            # Clean up the found number (remove spaces, dashes)
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 2: Look for Aadhaar numbers with keywords
    # This catches Aadhaar numbers that are labeled
    keyword_patterns = [
        # Various ways "Aadhaar" might be written followed by a number
        r'(?:aadhar|aadhaar|adhar|aadha+r|आधार)(?:\s*(?:card|number|no|id|#|:|नंबर|संख्या))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        r'(?:uid|unique\s+id)(?:\s*(?:number|no|#))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        # Looking for "No:" or "Number:" followed by what could be an Aadhaar
        r'(?:no|number|id)?\s*[:\.\-]\s*((?:\d[\d\s\.\-]*){12})'
    ]
    
    for pattern in keyword_patterns:
        matches = re.findall(pattern, cleaned_text.lower())
        if matches:
            # Clean up the found number
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 3: More aggressive - find any 12-digit sequence that could be an Aadhaar number
    # Use with caution as it might pick up other 12-digit numbers
    digit_sequences = re.findall(r'(?<!\d)(\d[\d\s\.\-]*\d)(?!\d)', cleaned_text)
    for seq in digit_sequences:
        digits_only = re.sub(r'[^\d]', '', seq)
        if len(digits_only) == 12:
            return digits_only
            
    return None

def process_text(text, keywords=[]):
    extracted_info = []
    
    # Special extraction for Aadhaar number
    aadhaar = extract_aadhaar_number(text)
    if aadhaar:
        # Format as XXXX-XXXX-XXXX
        formatted_aadhaar = f"{aadhaar[:4]}-{aadhaar[4:8]}-{aadhaar[8:]}"
        extracted_info.append({"Text": formatted_aadhaar, "Label": "Aadhar Card"})
    
    # Improve disease details extraction - handle multi-line content better
    disease_details_pattern = re.compile(r'(?:Disease Details|Details|Diagnosis Details|Clinical Details|Medical Details|Condition Details)[:;]\s*([\w\s,.;:()\-/\n]+)(?=\n\n|\n[A-Z]|$)', re.IGNORECASE)
    disease_details_matches = disease_details_pattern.search(text)
    if disease_details_matches:
        details = disease_details_matches.group(1).strip()
        # Clean up the details
        details = re.sub(r'\n+', ' ', details)
        details = re.sub(r'\s+', ' ', details)
        extracted_info.append({"Text": details, "Label": "Disease Details"})

    # Improve medicines extraction
    medicines_pattern = re.compile(r'(?:Medicines|Medications|Drugs|Prescriptions|Medicine List|Prescribed Medicines)[:;]\s*([\w\s,.;:()\-/\n]+)(?=\n\n|\n[A-Z]|$)', re.IGNORECASE)
    medicines_matches = medicines_pattern.search(text)
    if medicines_matches:
        medicines = medicines_matches.group(1).strip()
        # Clean up the medicines list
        medicines = re.sub(r'\n+', ' ', medicines)
        medicines = re.sub(r'\s+', ' ', medicines)
        extracted_info.append({"Text": medicines, "Label": "Medicines"})

    # Process other keywords as before
    for keyword in keywords:
        # Skip the fields we've already handled with specialized patterns
        if any(term in keyword.lower() for term in ["aadhar", "aadhaar", "disease details", "medicines", "medication"]):
            continue
            
        if keyword in ["Name:", "Customer Name", "Recipient", "Name", "Patient Name"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Father's Name:", "Father Name:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Gender:", "Sex:"]:
            pattern = re.compile(rf"{keyword}\s*([\w]+)")
        elif keyword in ["Blood Group:", "Blood:"]:
            pattern = re.compile(rf"{keyword}\s*([\w+-]+)")
        elif keyword in ["Address:", "Address"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s,-]+(?:\n[\w\s,-]+)*)")
        elif keyword in ["Hospital Name:", "Hospital"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Insurance ID:", "InsuranceID:", "Insurance Id", "Insurance"]:
            pattern = re.compile(rf"{keyword}\s*([\w\d-]+)")
        elif keyword in ["Appointment Time:", "Appointment", "Time:"]:
            pattern = re.compile(rf"{keyword}\s*([\d:]+(?:\s*[AP]M)?)")
        elif keyword in ["Phone:", "Phone Number:", "Contact:", "Mobile:"]:
            pattern = re.compile(rf"{keyword}\s*([\d\s-]+)")
        elif keyword in ["Amount:", "Amount", "Total:"]:
            pattern = re.compile(rf"{keyword}\s*([\d\.]+)")
        elif keyword in ["Disease Name:", "Disease:", "Diagnosis:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+(?:\n[\w\s]+)*)")
        elif keyword in ["Bed Type:", "Bed:"]:
            pattern = re.compile(rf"{keyword}\s*([\w\s]+)")
        elif keyword in ["Ventilation:", "Ventilator:"]:
            pattern = re.compile(rf"{keyword}\s*(Yes|No)")
        elif keyword in ["Other Charges:", "Charges:"]:
            pattern = re.compile(rf"{keyword}\s*([\d\.]+)")
        else:
            continue

        for match in pattern.finditer(text):
            extracted_text = match.group(1).strip()
            # Improved handling of multi-line text - take first line only for certain fields
            if keyword in ["Name:", "Father's Name:", "Gender:", "Blood Group:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Bed Type:", "Ventilation:", "Other Charges:"]:
                extracted_text = extracted_text.split("\n")[0].strip()
            extracted_info.append({"Text": extracted_text, "Label": keyword.replace(":", "")})

    return extracted_info

def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        # Insert document information into the 'patient_documents' table
        insert_doc_query = "INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)"
        doc_values = (insurance_id, file_path)
        cursor.execute(insert_doc_query, doc_values)

        # Insert patient details into the 'patient_details' table
        insert_patient_query = "INSERT INTO patient_details (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        name = next((item["Text"] for item in data if item["Label"] == "Name"), None)
        father_name = next((item["Text"] for item in data if item["Label"] == "Father's Name"), None)
        aadhar_card = next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None)
        gender = next((item["Text"] for item in data if item["Label"] == "Gender"), None)
        blood_group = next((item["Text"] for item in data if item["Label"] == "Blood Group"), None)
        address = next((item["Text"] for item in data if item["Label"] == "Address"), None)
        hospital_name = next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None)
        appointment_time = next((item["Text"] for item in data if item["Label"] == "Appointment Time"), None)
        phone_number = next((item["Text"] for item in data if item["Label"] == "Phone Number"), None)
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        disease_name = next((item["Text"] for item in data if item["Label"] == "Disease Name"), None)
        disease_details = next((item["Text"] for item in data if item["Label"] == "Disease Details"), None)
        medicines = next((item["Text"] for item in data if item["Label"] == "Medicines"), None)
        bed_type = next((item["Text"] for item in data if item["Label"] == "Bed Type"), None)
        ventilation = next((item["Text"] for item in data if item["Label"] == "Ventilation"), None)
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)

        patient_values = (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges)
        cursor.execute(insert_patient_query, patient_values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")

def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

It is extracting the adhaar number as well as medicines. But screwing up by adding unecessary details at the end of each expression.

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def extract_aadhaar_number(text):
    """
    Specialized function to extract Aadhaar numbers using multiple approaches
    """
    # Clean the text first - this can help with OCR errors
    cleaned_text = re.sub(r'\s+', ' ', text)
    
    # Method 1: Look for common Aadhaar number patterns (12 digits with or without separators)
    # This catches raw numbers that look like Aadhaar numbers
    aadhaar_patterns = [
        # Pattern with no separators - 12 consecutive digits
        r'(?<!\d)(\d{12})(?!\d)',
        # Pattern with space separators
        r'(\d{4}\s+\d{4}\s+\d{4})',
        # Pattern with dash separators
        r'(\d{4}-\d{4}-\d{4})',
        # Pattern with dot separators
        r'(\d{4}\.\d{4}\.\d{4})'
    ]
    
    for pattern in aadhaar_patterns:
        matches = re.findall(pattern, cleaned_text)
        if matches:
            # Clean up the found number (remove spaces, dashes)
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 2: Look for Aadhaar numbers with keywords
    # This catches Aadhaar numbers that are labeled
    keyword_patterns = [
        # Various ways "Aadhaar" might be written followed by a number
        r'(?:aadhar|aadhaar|adhar|aadha+r|आधार)(?:\s*(?:card|number|no|id|#|:|नंबर|संख्या))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        r'(?:uid|unique\s+id)(?:\s*(?:number|no|#))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        # Looking for "No:" or "Number:" followed by what could be an Aadhaar
        r'(?:no|number|id)?\s*[:\.\-]\s*((?:\d[\d\s\.\-]*){12})'
    ]
    
    for pattern in keyword_patterns:
        matches = re.findall(pattern, cleaned_text.lower())
        if matches:
            # Clean up the found number
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 3: More aggressive - find any 12-digit sequence that could be an Aadhaar number
    # Use with caution as it might pick up other 12-digit numbers
    digit_sequences = re.findall(r'(?<!\d)(\d[\d\s\.\-]*\d)(?!\d)', cleaned_text)
    for seq in digit_sequences:
        digits_only = re.sub(r'[^\d]', '', seq)
        if len(digits_only) == 12:
            return digits_only
            
    return None

def clean_extracted_field(text, field_type):
    """
    Cleans extracted text based on field type to remove common OCR artifacts
    and mislabeled content
    """
    # Convert to string in case we received another type
    text = str(text).strip()
    
    # Remove common label text that might be captured within the value
    unwanted_labels = [
        "Phone Number", "Contact", "Mobile", "Call",
        "Hospital Name", "Doctor", "Clinic", "MD", "Dr\\.",
        "Address", "Location", "Place", "Residence",
        "Insurance ID", "Policy Number", "Insurance",
        "Amount", "Total", "Fee", "Payment",
        "Disease", "Diagnosis", "Condition",
        "Medicines", "Medication", "Drugs", "Prescription"
    ]
    
    # For each unwanted label, try to remove it if it appears at the end
    for label in unwanted_labels:
        # Create pattern to match label at the end of the text (allowing for spaces)
        pattern = rf'\s*{re.escape(label)}$'
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # Remove common field separators
    text = re.sub(r'[:;|]$', '', text)
    
    # Clean up newlines and extra spaces
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Additional field-specific cleaning
    if field_type in ["Address"]:
        # Keep only relevant address information
        text = re.sub(r'\s*(?:Phone|Mobile|Contact|Email).*$', '', text, flags=re.IGNORECASE)
    
    elif field_type in ["Hospital Name"]:
        # Remove doctor references
        text = re.sub(r'\s*(?:Doctor|Dr\.|MD|Physician).*$', '', text, flags=re.IGNORECASE)
    
    elif field_type in ["Phone Number"]:
        # Keep only digits and basic formatting characters
        text = re.sub(r'[^\d+\-\s()]', '', text)
    
    return text.strip()

def extract_fields_with_boundaries(text):
    """
    Extract fields with improved boundary detection to prevent label bleed
    """
    extracted_info = []
    found_labels = set()
    
    # Dictionary of field patterns with better boundary detection
    field_patterns = {
        "Name": r'(?:Patient(?:\s*Name)?|Name|Patient)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Father|Gender|Blood|Aadhaar))',
        "Father's Name": r'(?:Father(?:[\'s]*\s*Name)?|Father)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Gender|Blood|Aadhaar))',
        "Gender": r'(?:Gender|Sex)[:;]?\s*(Male|Female|Other|M|F)(?=\n|$)',
        "Blood Group": r'(?:Blood(?:\s*Group)?)[:;]?\s*([ABO][+-]|AB[+-])(?=\n|$)',
        "Address": r'(?:Address|Location|Place|Residence)[:;]?\s*([\w\s,\.\-\/]+?)(?=\n|$|(?:Phone|Mobile|Contact|Email))',
        "Hospital Name": r'(?:Hospital(?:\s*Name)?|Clinic|Medical Center)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Doctor|Dr|MD|Address))',
        "Insurance ID": r'(?:Insurance(?:\s*(?:ID|Number|No))?|Policy(?:\s*Number)?)[:;]?\s*([\w\d\-]+?)(?=\n|$)',
        "Appointment Time": r'(?:Appointment(?:\s*Time)?|Time|Schedule)[:;]?\s*([\d:]+(?:\s*[AP]M)?)(?=\n|$)',
        "Phone Number": r'(?:Phone(?:\s*Number)?|Mobile|Contact|Cell)[:;]?\s*([\d\s\+\-\(\)]+?)(?=\n|$)',
        "Amount": r'(?:Amount|Total|Cost|Fee|Charges)[:;]?\s*([\d\.]+?)(?=\n|$|Rs|\$|₹)',
        "Disease Name": r'(?:Disease(?:\s*Name)?|Diagnosis|Condition|Ailment)[:;]?\s*([\w\s]+?)(?=\n|$|(?:Disease Details|Symptoms|Treatment))',
        "Disease Details": r'(?:Disease(?:\s*Details)?|Details|Diagnosis Details|Clinical Details|Symptoms)[:;]?\s*([\w\s,\.;\(\)\-\/]+?)(?=\n\n|\n(?:Medicines|Medications|Drugs)|$)',
        "Medicines": r'(?:Medicines|Medications|Drugs|Prescriptions|Medicine List)[:;]?\s*([\w\s,\.;\(\)\-\/]+?)(?=\n\n|\n(?:Bed|Ventilation|Amount|Charges)|$)',
        "Bed Type": r'(?:Bed(?:\s*Type)?)[:;]?\s*([\w\s]+?)(?=\n|$)',
        "Ventilation": r'(?:Ventilation|Ventilator|Oxygen)[:;]?\s*(Yes|No|Required|Not Required)(?=\n|$)',
        "Other Charges": r'(?:Other(?:\s*Charges)?|Additional(?:\s*Charges)?|Extra)[:;]?\s*([\d\.]+?)(?=\n|$|Rs|\$|₹)'
    }
    
    # 1. First pass: Extract Aadhaar number with dedicated function
    aadhaar = extract_aadhaar_number(text)
    if aadhaar:
        formatted_aadhaar = f"{aadhaar[:4]}-{aadhaar[4:8]}-{aadhaar[8:]}"
        extracted_info.append({"Text": formatted_aadhaar, "Label": "Aadhar Card"})
        found_labels.add("Aadhar Card")
    
    # 2. Second pass: Extract other fields with improved boundary detection
    for label, pattern in field_patterns.items():
        if label in found_labels:
            continue
            
        matches = re.search(pattern, text, re.IGNORECASE)
        if matches:
            extracted_text = matches.group(1).strip()
            # Clean the extracted text to remove potential label contamination
            cleaned_text = clean_extracted_field(extracted_text, label)
            
            # Only add if we have meaningful content
            if cleaned_text and len(cleaned_text) > 0:
                extracted_info.append({"Text": cleaned_text, "Label": label})
                found_labels.add(label)
    
    # 3. Third pass: Look for unlabeled numbers that might be specific fields
    if "Phone Number" not in found_labels:
        # Look for potential phone numbers (10-digit sequences)
        phone_matches = re.search(r'(?<!\d)(\d{10})(?!\d)', text)
        if phone_matches:
            extracted_info.append({"Text": phone_matches.group(1), "Label": "Phone Number"})
            found_labels.add("Phone Number")
    
    # Look for Appendicitis or other common conditions if disease name not found
    if "Disease Name" not in found_labels:
        common_diseases = ["appendicitis", "diabetes", "hypertension", "cancer", "fracture", "pneumonia"]
        for disease in common_diseases:
            if re.search(rf'\b{disease}\b', text, re.IGNORECASE):
                extracted_info.append({"Text": disease.capitalize(), "Label": "Disease Name"})
                found_labels.add("Disease Name")
                break
    
    return extracted_info

def process_text(text, keywords=[]):
    """
    Main processing function that combines extraction methods
    """
    # Get fields using improved boundary detection
    extracted_info = extract_fields_with_boundaries(text)
    
    # For backward compatibility, still use keyword-based extraction for any missing fields
    found_labels = {item["Label"] for item in extracted_info}
    
    for keyword in keywords:
        # Skip keywords for fields we already found
        label = keyword.replace(":", "").strip()
        if any(label in existing for existing in found_labels):
            continue
            
        # Simple keyword-based extraction as fallback
        pattern = re.compile(rf"{re.escape(keyword)}\s*([\w\s\d\.\-]+?)(?=\n|$)", re.IGNORECASE)
        match = pattern.search(text)
        if match:
            extracted_text = match.group(1).strip()
            cleaned_text = clean_extracted_field(extracted_text, label)
            
            if cleaned_text and len(cleaned_text) > 0:
                extracted_info.append({"Text": cleaned_text, "Label": label})
                found_labels.add(label)
    
    return extracted_info

# The extract_aadhaar_number function from previous solution should be included here
def extract_aadhaar_number(text):
    """
    Specialized function to extract Aadhaar numbers using multiple approaches
    """
    # Clean the text first - this can help with OCR errors
    cleaned_text = re.sub(r'\s+', ' ', text)
    
    # Method 1: Look for common Aadhaar number patterns (12 digits with or without separators)
    aadhaar_patterns = [
        # Pattern with no separators - 12 consecutive digits
        r'(?<!\d)(\d{12})(?!\d)',
        # Pattern with space separators
        r'(\d{4}\s+\d{4}\s+\d{4})',
        # Pattern with dash separators
        r'(\d{4}-\d{4}-\d{4})',
        # Pattern with dot separators
        r'(\d{4}\.\d{4}\.\d{4})'
    ]
    
    for pattern in aadhaar_patterns:
        matches = re.findall(pattern, cleaned_text)
        if matches:
            # Clean up the found number (remove spaces, dashes)
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 2: Look for Aadhaar numbers with keywords
    keyword_patterns = [
        # Various ways "Aadhaar" might be written followed by a number
        r'(?:aadhar|aadhaar|adhar|aadha+r|आधार)(?:\s*(?:card|number|no|id|#|:|नंबर|संख्या))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        r'(?:uid|unique\s+id)(?:\s*(?:number|no|#))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        # Looking for "No:" or "Number:" followed by what could be an Aadhaar
        r'(?:no|number|id)?\s*[:\.\-]\s*((?:\d[\d\s\.\-]*){12})'
    ]
    
    for pattern in keyword_patterns:
        matches = re.findall(pattern, cleaned_text.lower())
        if matches:
            # Clean up the found number
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 3: More aggressive - find any 12-digit sequence that could be an Aadhaar number
    digit_sequences = re.findall(r'(?<!\d)(\d[\d\s\.\-]*\d)(?!\d)', cleaned_text)
    for seq in digit_sequences:
        digits_only = re.sub(r'[^\d]', '', seq)
        if len(digits_only) == 12:
            return digits_only
            
    return None
def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        # Insert document information into the 'patient_documents' table
        insert_doc_query = "INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)"
        doc_values = (insurance_id, file_path)
        cursor.execute(insert_doc_query, doc_values)

        # Insert patient details into the 'patient_details' table
        insert_patient_query = "INSERT INTO patient_details (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        name = next((item["Text"] for item in data if item["Label"] == "Name"), None)
        father_name = next((item["Text"] for item in data if item["Label"] == "Father's Name"), None)
        aadhar_card = next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None)
        gender = next((item["Text"] for item in data if item["Label"] == "Gender"), None)
        blood_group = next((item["Text"] for item in data if item["Label"] == "Blood Group"), None)
        address = next((item["Text"] for item in data if item["Label"] == "Address"), None)
        hospital_name = next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None)
        appointment_time = next((item["Text"] for item in data if item["Label"] == "Appointment Time"), None)
        phone_number = next((item["Text"] for item in data if item["Label"] == "Phone Number"), None)
        
        # Clean the amount value to remove currency symbols and other non-numeric characters
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        if amount:
            # Extract only the numeric part (keep decimal point)
            amount = re.sub(r'[^\d.]', '', amount)
            # If there's still content after cleaning, convert to float
            if amount:
                try:
                    amount = float(amount)
                except ValueError:
                    amount = None
        
        disease_name = next((item["Text"] for item in data if item["Label"] == "Disease Name"), None)
        disease_details = next((item["Text"] for item in data if item["Label"] == "Disease Details"), None)
        medicines = next((item["Text"] for item in data if item["Label"] == "Medicines"), None)
        bed_type = next((item["Text"] for item in data if item["Label"] == "Bed Type"), None)
        ventilation = next((item["Text"] for item in data if item["Label"] == "Ventilation"), None)
        
        # Clean the other_charges value similarly
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)
        if other_charges:
            other_charges = re.sub(r'[^\d.]', '', other_charges)
            if other_charges:
                try:
                    other_charges = float(other_charges)
                except ValueError:
                    other_charges = None

        patient_values = (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges)
        cursor.execute(insert_patient_query, patient_values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")
def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

The above code is mostly working fine.

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def extract_aadhaar_number(text):
    """
    Specialized function to extract Aadhaar numbers using multiple approaches
    """
    # Clean the text first - this can help with OCR errors
    cleaned_text = re.sub(r'\s+', ' ', text)
    
    # Method 1: Look for common Aadhaar number patterns (12 digits with or without separators)
    # This catches raw numbers that look like Aadhaar numbers
    aadhaar_patterns = [
        # Pattern with no separators - 12 consecutive digits
        r'(?<!\d)(\d{12})(?!\d)',
        # Pattern with space separators
        r'(\d{4}\s+\d{4}\s+\d{4})',
        # Pattern with dash separators
        r'(\d{4}-\d{4}-\d{4})',
        # Pattern with dot separators
        r'(\d{4}\.\d{4}\.\d{4})'
    ]
    
    for pattern in aadhaar_patterns:
        matches = re.findall(pattern, cleaned_text)
        if matches:
            # Clean up the found number (remove spaces, dashes)
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 2: Look for Aadhaar numbers with keywords
    # This catches Aadhaar numbers that are labeled
    keyword_patterns = [
        # Various ways "Aadhaar" might be written followed by a number
        r'(?:aadhar|aadhaar|adhar|aadha+r|आधार)(?:\s*(?:card|number|no|id|#|:|नंबर|संख्या))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        r'(?:uid|unique\s+id)(?:\s*(?:number|no|#))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        # Looking for "No:" or "Number:" followed by what could be an Aadhaar
        r'(?:no|number|id)?\s*[:\.\-]\s*((?:\d[\d\s\.\-]*){12})'
    ]
    
    for pattern in keyword_patterns:
        matches = re.findall(pattern, cleaned_text.lower())
        if matches:
            # Clean up the found number
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 3: More aggressive - find any 12-digit sequence that could be an Aadhaar number
    # Use with caution as it might pick up other 12-digit numbers
    digit_sequences = re.findall(r'(?<!\d)(\d[\d\s\.\-]*\d)(?!\d)', cleaned_text)
    for seq in digit_sequences:
        digits_only = re.sub(r'[^\d]', '', seq)
        if len(digits_only) == 12:
            return digits_only
            
    return None

def clean_extracted_field(text, field_type):
    """
    Cleans extracted text based on field type to remove common OCR artifacts
    and mislabeled content
    """
    # Convert to string in case we received another type
    text = str(text).strip()
    
    # Remove common label text that might be captured within the value
    unwanted_labels = [
        "Phone Number", "Contact", "Mobile", "Call",
        "Hospital Name", "Doctor", "Clinic", "MD", "Dr\\.",
        "Address", "Location", "Place", "Residence",
        "Insurance ID", "Policy Number", "Insurance",
        "Amount", "Total", "Fee", "Payment",
        "Disease", "Diagnosis", "Condition",
        "Medicines", "Medication", "Drugs", "Prescription"
    ]
    
    # For each unwanted label, try to remove it if it appears at the end
    for label in unwanted_labels:
        # Create pattern to match label at the end of the text (allowing for spaces)
        pattern = rf'\s*{re.escape(label)}$'
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # Remove common field separators
    text = re.sub(r'[:;|]$', '', text)
    
    # Clean up newlines and extra spaces
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Additional field-specific cleaning
    if field_type in ["Address"]:
        # Keep only relevant address information
        text = re.sub(r'\s*(?:Phone|Mobile|Contact|Email).*$', '', text, flags=re.IGNORECASE)
    
    elif field_type in ["Hospital Name"]:
        # Remove doctor references
        text = re.sub(r'\s*(?:Doctor|Dr\.|MD|Physician).*$', '', text, flags=re.IGNORECASE)
    
    elif field_type in ["Phone Number"]:
        # Keep only digits and basic formatting characters
        text = re.sub(r'[^\d+\-\s()]', '', text)
    
    return text.strip()

def extract_fields_with_boundaries(text):
    """
    Extract fields with improved boundary detection to prevent label bleed
    """
    extracted_info = []
    found_labels = set()
    
    # Dictionary of field patterns with better boundary detection
    field_patterns = {
        "Name": r'(?:Patient(?:\s*Name)?|Name|Patient)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Father|Gender|Blood|Aadhaar))',
        "Father's Name": r'(?:Father(?:[\'s]*\s*Name)?|Father)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Gender|Blood|Aadhaar))',
        "Gender": r'(?:Gender|Sex)[:;]?\s*(Male|Female|Other|M|F)(?=\n|$)',
        "Blood Group": r'(?:Blood(?:\s*Group)?)[:;]?\s*([ABO][+-]|AB[+-])(?=\n|$)',
        "Address": r'(?:Address|Location|Place|Residence)[:;]?\s*([\w\s,\.\-\/]+?)(?=\n|$|(?:Phone|Mobile|Contact|Email))',
        "Hospital Name": r'(?:Hospital(?:\s*Name)?|Clinic|Medical Center)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Doctor|Dr|MD|Address))',
        "Insurance ID": r'(?:Insurance(?:\s*(?:ID|Number|No))?|Policy(?:\s*Number)?)[:;]?\s*([\w\d\-]+?)(?=\n|$)',
        "Appointment Time": r'(?:Appointment(?:\s*Time)?|Time|Schedule)[:;]?\s*([\d:]+(?:\s*[AP]M)?)(?=\n|$)',
        "Phone Number": r'(?:Phone(?:\s*Number)?|Mobile|Contact|Cell)[:;]?\s*([\d\s\+\-\(\)]+?)(?=\n|$)',
        "Amount": r'(?:Amount|Total|Cost|Fee|Charges)[:;]?\s*([\d\.]+?)(?=\n|$|Rs|\$|₹)',
        "Disease Name": r'(?:Disease(?:\s*Name)?|Diagnosis|Condition|Ailment)[:;]?\s*([\w\s]+?)(?=\n|$|(?:Disease Details|Symptoms|Treatment))',
        "Disease Details": r'(?:Disease(?:\s*Details)?|Details|Diagnosis Details|Clinical Details|Symptoms)[:;]?\s*([\w\s,\.;\(\)\-\/]+?)(?=\n\n|\n(?:Medicines|Medications|Drugs)|$)',
        "Medicines": r'(?:Medicines|Medications|Drugs|Prescriptions|Medicine List)[:;]?\s*([\w\s,\.;\(\)\-\/]+?)(?=\n\n|\n(?:Bed|Ventilation|Amount|Charges)|$)',
        "Bed Type": r'(?:Bed(?:\s*Type)?)[:;]?\s*([\w\s]+?)(?=\n|$)',
        "Ventilation": r'(?:Ventilation|Ventilator|Oxygen)[:;]?\s*(Yes|No|Required|Not Required)(?=\n|$)',
        "Other Charges": r'(?:Other(?:\s*Charges)?|Additional(?:\s*Charges)?|Extra)[:;]?\s*([\d\.]+?)(?=\n|$|Rs|\$|₹)'
    }
    
    # 1. First pass: Extract Aadhaar number with dedicated function
    aadhaar = extract_aadhaar_number(text)
    if aadhaar:
        formatted_aadhaar = f"{aadhaar[:4]}-{aadhaar[4:8]}-{aadhaar[8:]}"
        extracted_info.append({"Text": formatted_aadhaar, "Label": "Aadhar Card"})
        found_labels.add("Aadhar Card")
    
    # 2. Second pass: Extract other fields with improved boundary detection
    for label, pattern in field_patterns.items():
        if label in found_labels:
            continue
            
        matches = re.search(pattern, text, re.IGNORECASE)
        if matches:
            extracted_text = matches.group(1).strip()
            # Clean the extracted text to remove potential label contamination
            cleaned_text = clean_extracted_field(extracted_text, label)
            
            # Only add if we have meaningful content
            if cleaned_text and len(cleaned_text) > 0:
                extracted_info.append({"Text": cleaned_text, "Label": label})
                found_labels.add(label)
    
    # 3. Third pass: Look for unlabeled numbers that might be specific fields
    if "Phone Number" not in found_labels:
        # Look for potential phone numbers (10-digit sequences)
        phone_matches = re.search(r'(?<!\d)(\d{10})(?!\d)', text)
        if phone_matches:
            extracted_info.append({"Text": phone_matches.group(1), "Label": "Phone Number"})
            found_labels.add("Phone Number")
    
    # Look for Appendicitis or other common conditions if disease name not found
    if "Disease Name" not in found_labels:
        common_diseases = ["appendicitis", "diabetes", "hypertension", "cancer", "fracture", "pneumonia"]
        for disease in common_diseases:
            if re.search(rf'\b{disease}\b', text, re.IGNORECASE):
                extracted_info.append({"Text": disease.capitalize(), "Label": "Disease Name"})
                found_labels.add("Disease Name")
                break
    
    return extracted_info

def process_text(text, keywords=[]):
    """
    Main processing function that combines extraction methods
    """
    # Get fields using improved boundary detection
    extracted_info = extract_fields_with_boundaries(text)
    
    # For backward compatibility, still use keyword-based extraction for any missing fields
    found_labels = {item["Label"] for item in extracted_info}
    
    for keyword in keywords:
        # Skip keywords for fields we already found
        label = keyword.replace(":", "").strip()
        if any(label in existing for existing in found_labels):
            continue
            
        # Simple keyword-based extraction as fallback
        pattern = re.compile(rf"{re.escape(keyword)}\s*([\w\s\d\.\-]+?)(?=\n|$)", re.IGNORECASE)
        match = pattern.search(text)
        if match:
            extracted_text = match.group(1).strip()
            cleaned_text = clean_extracted_field(extracted_text, label)
            
            if cleaned_text and len(cleaned_text) > 0:
                extracted_info.append({"Text": cleaned_text, "Label": label})
                found_labels.add(label)
    
    return extracted_info

# The extract_aadhaar_number function from previous solution should be included here
def extract_aadhaar_number(text):
    """
    Specialized function to extract Aadhaar numbers using multiple approaches
    """
    # Clean the text first - this can help with OCR errors
    cleaned_text = re.sub(r'\s+', ' ', text)
    
    # Method 1: Look for common Aadhaar number patterns (12 digits with or without separators)
    aadhaar_patterns = [
        # Pattern with no separators - 12 consecutive digits
        r'(?<!\d)(\d{12})(?!\d)',
        # Pattern with space separators
        r'(\d{4}\s+\d{4}\s+\d{4})',
        # Pattern with dash separators
        r'(\d{4}-\d{4}-\d{4})',
        # Pattern with dot separators
        r'(\d{4}\.\d{4}\.\d{4})'
    ]
    
    for pattern in aadhaar_patterns:
        matches = re.findall(pattern, cleaned_text)
        if matches:
            # Clean up the found number (remove spaces, dashes)
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 2: Look for Aadhaar numbers with keywords
    keyword_patterns = [
        # Various ways "Aadhaar" might be written followed by a number
        r'(?:aadhar|aadhaar|adhar|aadha+r|आधार)(?:\s*(?:card|number|no|id|#|:|नंबर|संख्या))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        r'(?:uid|unique\s+id)(?:\s*(?:number|no|#))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        # Looking for "No:" or "Number:" followed by what could be an Aadhaar
        r'(?:no|number|id)?\s*[:\.\-]\s*((?:\d[\d\s\.\-]*){12})'
    ]
    
    for pattern in keyword_patterns:
        matches = re.findall(pattern, cleaned_text.lower())
        if matches:
            # Clean up the found number
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 3: More aggressive - find any 12-digit sequence that could be an Aadhaar number
    digit_sequences = re.findall(r'(?<!\d)(\d[\d\s\.\-]*\d)(?!\d)', cleaned_text)
    for seq in digit_sequences:
        digits_only = re.sub(r'[^\d]', '', seq)
        if len(digits_only) == 12:
            return digits_only
            
    return None
def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        # Insert document information into the 'patient_documents' table
        insert_doc_query = "INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)"
        doc_values = (insurance_id, file_path)
        cursor.execute(insert_doc_query, doc_values)

        # Insert patient details into the 'patient_details' table
        insert_patient_query = "INSERT INTO patient_details (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        name = next((item["Text"] for item in data if item["Label"] == "Name"), None)
        father_name = next((item["Text"] for item in data if item["Label"] == "Father's Name"), None)
        aadhar_card = next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None)
        gender = next((item["Text"] for item in data if item["Label"] == "Gender"), None)
        blood_group = next((item["Text"] for item in data if item["Label"] == "Blood Group"), None)
        address = next((item["Text"] for item in data if item["Label"] == "Address"), None)
        hospital_name = next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None)
        appointment_time = next((item["Text"] for item in data if item["Label"] == "Appointment Time"), None)
        phone_number = next((item["Text"] for item in data if item["Label"] == "Phone Number"), None)
        
        # Clean the amount value to remove currency symbols and other non-numeric characters
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        if amount:
            # Extract only the numeric part (keep decimal point)
            amount = re.sub(r'[^\d.]', '', amount)
            # If there's still content after cleaning, convert to float
            if amount:
                try:
                    amount = float(amount)
                except ValueError:
                    amount = None
        
        disease_name = next((item["Text"] for item in data if item["Label"] == "Disease Name"), None)
        disease_details = next((item["Text"] for item in data if item["Label"] == "Disease Details"), None)
        medicines = next((item["Text"] for item in data if item["Label"] == "Medicines"), None)
        bed_type = next((item["Text"] for item in data if item["Label"] == "Bed Type"), None)
        
        # Handle ventilation field - convert N/A, NULL, etc. to "No"
        ventilation = next((item["Text"] for item in data if item["Label"] == "Ventilation"), None)
        if ventilation is None or ventilation.upper() in ["N/A", "NA", "NULL", ""]:
            ventilation = "No"
        
        # Clean the other_charges value similarly
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)
        if other_charges:
            other_charges = re.sub(r'[^\d.]', '', other_charges)
            if other_charges:
                try:
                    other_charges = float(other_charges)
                except ValueError:
                    other_charges = None

        patient_values = (insurance_id, name, father_name, aadhar_card, gender, blood_group, address, hospital_name, appointment_time, phone_number, amount, disease_name, disease_details, medicines, bed_type, ventilation, other_charges)
        cursor.execute(insert_patient_query, patient_values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")
def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

The ventilation part is fucking up. So we have decided to delete the ventilation column only.

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def extract_aadhaar_number(text):
    """
    Specialized function to extract Aadhaar numbers using multiple approaches
    """
    # Clean the text first - this can help with OCR errors
    cleaned_text = re.sub(r'\s+', ' ', text)
    
    # Method 1: Look for common Aadhaar number patterns (12 digits with or without separators)
    # This catches raw numbers that look like Aadhaar numbers
    aadhaar_patterns = [
        # Pattern with no separators - 12 consecutive digits
        r'(?<!\d)(\d{12})(?!\d)',
        # Pattern with space separators
        r'(\d{4}\s+\d{4}\s+\d{4})',
        # Pattern with dash separators
        r'(\d{4}-\d{4}-\d{4})',
        # Pattern with dot separators
        r'(\d{4}\.\d{4}\.\d{4})'
    ]
    
    for pattern in aadhaar_patterns:
        matches = re.findall(pattern, cleaned_text)
        if matches:
            # Clean up the found number (remove spaces, dashes)
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 2: Look for Aadhaar numbers with keywords
    # This catches Aadhaar numbers that are labeled
    keyword_patterns = [
        # Various ways "Aadhaar" might be written followed by a number
        r'(?:aadhar|aadhaar|adhar|aadha+r|आधार)(?:\s*(?:card|number|no|id|#|:|नंबर|संख्या))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        r'(?:uid|unique\s+id)(?:\s*(?:number|no|#))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        # Looking for "No:" or "Number:" followed by what could be an Aadhaar
        r'(?:no|number|id)?\s*[:\.\-]\s*((?:\d[\d\s\.\-]*){12})'
    ]
    
    for pattern in keyword_patterns:
        matches = re.findall(pattern, cleaned_text.lower())
        if matches:
            # Clean up the found number
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 3: More aggressive - find any 12-digit sequence that could be an Aadhaar number
    # Use with caution as it might pick up other 12-digit numbers
    digit_sequences = re.findall(r'(?<!\d)(\d[\d\s\.\-]*\d)(?!\d)', cleaned_text)
    for seq in digit_sequences:
        digits_only = re.sub(r'[^\d]', '', seq)
        if len(digits_only) == 12:
            return digits_only
            
    return None

def clean_extracted_field(text, field_type):
    """
    Cleans extracted text based on field type to remove common OCR artifacts
    and mislabeled content
    """
    # Convert to string in case we received another type
    text = str(text).strip()
    
    # Remove common label text that might be captured within the value
    unwanted_labels = [
        "Phone Number", "Contact", "Mobile", "Call",
        "Hospital Name", "Doctor", "Clinic", "MD", "Dr\\.",
        "Address", "Location", "Place", "Residence",
        "Insurance ID", "Policy Number", "Insurance",
        "Amount", "Total", "Fee", "Payment",
        "Disease", "Diagnosis", "Condition",
        "Medicines", "Medication", "Drugs", "Prescription"
    ]
    
    # For each unwanted label, try to remove it if it appears at the end
    for label in unwanted_labels:
        # Create pattern to match label at the end of the text (allowing for spaces)
        pattern = rf'\s*{re.escape(label)}$'
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # Remove common field separators
    text = re.sub(r'[:;|]$', '', text)
    
    # Clean up newlines and extra spaces
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Additional field-specific cleaning
    if field_type in ["Address"]:
        # Keep only relevant address information
        text = re.sub(r'\s*(?:Phone|Mobile|Contact|Email).*$', '', text, flags=re.IGNORECASE)
    
    elif field_type in ["Hospital Name"]:
        # Remove doctor references
        text = re.sub(r'\s*(?:Doctor|Dr\.|MD|Physician).*$', '', text, flags=re.IGNORECASE)
    
    elif field_type in ["Phone Number"]:
        # Keep only digits and basic formatting characters
        text = re.sub(r'[^\d+\-\s()]', '', text)
    
    return text.strip()

def extract_fields_with_boundaries(text):
    """
    Extract fields with improved boundary detection to prevent label bleed
    """
    extracted_info = []
    found_labels = set()
    
    # Dictionary of field patterns with better boundary detection
    field_patterns = {
        "Name": r'(?:Patient(?:\s*Name)?|Name|Patient)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Father|Gender|Blood|Aadhaar))',
        "Father's Name": r'(?:Father(?:[\'s]*\s*Name)?|Father)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Gender|Blood|Aadhaar))',
        "Gender": r'(?:Gender|Sex)[:;]?\s*(Male|Female|Other|M|F)(?=\n|$)',
        "Blood Group": r'(?:Blood(?:\s*Group)?)[:;]?\s*([ABO][+-]|AB[+-])(?=\n|$)',
        "Address": r'(?:Address|Location|Place|Residence)[:;]?\s*([\w\s,\.\-\/]+?)(?=\n|$|(?:Phone|Mobile|Contact|Email))',
        "Hospital Name": r'(?:Hospital(?:\s*Name)?|Clinic|Medical Center)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Doctor|Dr|MD|Address))',
        "Insurance ID": r'(?:Insurance(?:\s*(?:ID|Number|No))?|Policy(?:\s*Number)?)[:;]?\s*([\w\d\-]+?)(?=\n|$)',
        "Appointment Time": r'(?:Appointment(?:\s*Time)?|Time|Schedule)[:;]?\s*([\d:]+(?:\s*[AP]M)?)(?=\n|$)',
        "Phone Number": r'(?:Phone(?:\s*Number)?|Mobile|Contact|Cell)[:;]?\s*([\d\s\+\-\(\)]+?)(?=\n|$)',
        "Amount": r'(?:Amount|Total|Cost|Fee|Charges)[:;]?\s*([\d\.]+?)(?=\n|$|Rs|\$|₹)',
        "Disease Name": r'(?:Disease(?:\s*Name)?|Diagnosis|Condition|Ailment)[:;]?\s*([\w\s]+?)(?=\n|$|(?:Disease Details|Symptoms|Treatment))',
        "Disease Details": r'(?:Disease(?:\s*Details)?|Details|Diagnosis Details|Clinical Details|Symptoms)[:;]?\s*([\w\s,\.;\(\)\-\/]+?)(?=\n\n|\n(?:Medicines|Medications|Drugs)|$)',
        "Medicines": r'(?:Medicines|Medications|Drugs|Prescriptions|Medicine List)[:;]?\s*([\w\s,\.;\(\)\-\/]+?)(?=\n\n|\n(?:Bed|Ventilation|Amount|Charges)|$)',
        "Bed Type": r'(?:Bed(?:\s*Type)?)[:;]?\s*([\w\s]+?)(?=\n|$)',
        "Ventilation": r'(?:Ventilation|Ventilator|Oxygen)[:;]?\s*(Yes|No|Required|Not Required)(?=\n|$)',
        "Other Charges": r'(?:Other(?:\s*Charges)?|Additional(?:\s*Charges)?|Extra)[:;]?\s*([\d\.]+?)(?=\n|$|Rs|\$|₹)'
    }
    
    # 1. First pass: Extract Aadhaar number with dedicated function
    aadhaar = extract_aadhaar_number(text)
    if aadhaar:
        formatted_aadhaar = f"{aadhaar[:4]}-{aadhaar[4:8]}-{aadhaar[8:]}"
        extracted_info.append({"Text": formatted_aadhaar, "Label": "Aadhar Card"})
        found_labels.add("Aadhar Card")
    
    # 2. Second pass: Extract other fields with improved boundary detection
    for label, pattern in field_patterns.items():
        if label in found_labels:
            continue
            
        matches = re.search(pattern, text, re.IGNORECASE)
        if matches:
            extracted_text = matches.group(1).strip()
            # Clean the extracted text to remove potential label contamination
            cleaned_text = clean_extracted_field(extracted_text, label)
            
            # Only add if we have meaningful content
            if cleaned_text and len(cleaned_text) > 0:
                extracted_info.append({"Text": cleaned_text, "Label": label})
                found_labels.add(label)
    
    # 3. Third pass: Look for unlabeled numbers that might be specific fields
    if "Phone Number" not in found_labels:
        # Look for potential phone numbers (10-digit sequences)
        phone_matches = re.search(r'(?<!\d)(\d{10})(?!\d)', text)
        if phone_matches:
            extracted_info.append({"Text": phone_matches.group(1), "Label": "Phone Number"})
            found_labels.add("Phone Number")
    
    # Look for Appendicitis or other common conditions if disease name not found
    if "Disease Name" not in found_labels:
        common_diseases = ["appendicitis", "diabetes", "hypertension", "cancer", "fracture", "pneumonia"]
        for disease in common_diseases:
            if re.search(rf'\b{disease}\b', text, re.IGNORECASE):
                extracted_info.append({"Text": disease.capitalize(), "Label": "Disease Name"})
                found_labels.add("Disease Name")
                break
    
    return extracted_info

def process_text(text, keywords=[]):
    """
    Main processing function that combines extraction methods
    """
    # Get fields using improved boundary detection
    extracted_info = extract_fields_with_boundaries(text)
    
    # For backward compatibility, still use keyword-based extraction for any missing fields
    found_labels = {item["Label"] for item in extracted_info}
    
    for keyword in keywords:
        # Skip keywords for fields we already found
        label = keyword.replace(":", "").strip()
        if any(label in existing for existing in found_labels):
            continue
            
        # Simple keyword-based extraction as fallback
        pattern = re.compile(rf"{re.escape(keyword)}\s*([\w\s\d\.\-]+?)(?=\n|$)", re.IGNORECASE)
        match = pattern.search(text)
        if match:
            extracted_text = match.group(1).strip()
            cleaned_text = clean_extracted_field(extracted_text, label)
            
            if cleaned_text and len(cleaned_text) > 0:
                extracted_info.append({"Text": cleaned_text, "Label": label})
                found_labels.add(label)
    
    return extracted_info

# The extract_aadhaar_number function from previous solution should be included here
def extract_aadhaar_number(text):
    """
    Specialized function to extract Aadhaar numbers using multiple approaches
    """
    # Clean the text first - this can help with OCR errors
    cleaned_text = re.sub(r'\s+', ' ', text)
    
    # Method 1: Look for common Aadhaar number patterns (12 digits with or without separators)
    aadhaar_patterns = [
        # Pattern with no separators - 12 consecutive digits
        r'(?<!\d)(\d{12})(?!\d)',
        # Pattern with space separators
        r'(\d{4}\s+\d{4}\s+\d{4})',
        # Pattern with dash separators
        r'(\d{4}-\d{4}-\d{4})',
        # Pattern with dot separators
        r'(\d{4}\.\d{4}\.\d{4})'
    ]
    
    for pattern in aadhaar_patterns:
        matches = re.findall(pattern, cleaned_text)
        if matches:
            # Clean up the found number (remove spaces, dashes)
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 2: Look for Aadhaar numbers with keywords
    keyword_patterns = [
        # Various ways "Aadhaar" might be written followed by a number
        r'(?:aadhar|aadhaar|adhar|aadha+r|आधार)(?:\s*(?:card|number|no|id|#|:|नंबर|संख्या))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        r'(?:uid|unique\s+id)(?:\s*(?:number|no|#))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        # Looking for "No:" or "Number:" followed by what could be an Aadhaar
        r'(?:no|number|id)?\s*[:\.\-]\s*((?:\d[\d\s\.\-]*){12})'
    ]
    
    for pattern in keyword_patterns:
        matches = re.findall(pattern, cleaned_text.lower())
        if matches:
            # Clean up the found number
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 3: More aggressive - find any 12-digit sequence that could be an Aadhaar number
    digit_sequences = re.findall(r'(?<!\d)(\d[\d\s\.\-]*\d)(?!\d)', cleaned_text)
    for seq in digit_sequences:
        digits_only = re.sub(r'[^\d]', '', seq)
        if len(digits_only) == 12:
            return digits_only
            
    return None
def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        # Insert document information into the 'patient_documents' table
        insert_doc_query = "INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)"
        doc_values = (insurance_id, file_path)
        cursor.execute(insert_doc_query, doc_values)

        # Modified query to exclude the ventilation column
        insert_patient_query = """
        INSERT INTO patient_details 
        (insurance_id, name, father_name, aadhar_card, gender, blood_group, 
        address, hospital_name, appointment_time, phone_number, amount, 
        disease_name, disease_details, medicines, bed_type, other_charges) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        name = next((item["Text"] for item in data if item["Label"] == "Name"), None)
        father_name = next((item["Text"] for item in data if item["Label"] == "Father's Name"), None)
        aadhar_card = next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None)
        gender = next((item["Text"] for item in data if item["Label"] == "Gender"), None)
        blood_group = next((item["Text"] for item in data if item["Label"] == "Blood Group"), None)
        address = next((item["Text"] for item in data if item["Label"] == "Address"), None)
        hospital_name = next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None)
        appointment_time = next((item["Text"] for item in data if item["Label"] == "Appointment Time"), None)
        phone_number = next((item["Text"] for item in data if item["Label"] == "Phone Number"), None)
        
        # Clean the amount value
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        if amount:
            amount = re.sub(r'[^\d.]', '', amount)
            if amount:
                try:
                    amount = float(amount)
                except ValueError:
                    amount = None
        
        disease_name = next((item["Text"] for item in data if item["Label"] == "Disease Name"), None)
        disease_details = next((item["Text"] for item in data if item["Label"] == "Disease Details"), None)
        medicines = next((item["Text"] for item in data if item["Label"] == "Medicines"), None)
        bed_type = next((item["Text"] for item in data if item["Label"] == "Bed Type"), None)
        
        # Clean other_charges
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)
        if other_charges:
            other_charges = re.sub(r'[^\d.]', '', other_charges)
            if other_charges:
                try:
                    other_charges = float(other_charges)
                except ValueError:
                    other_charges = None

        # Note: ventilation is removed from the values tuple
        patient_values = (insurance_id, name, father_name, aadhar_card, gender, blood_group, 
                          address, hospital_name, appointment_time, phone_number, amount, 
                          disease_name, disease_details, medicines, bed_type, other_charges)
        
        cursor.execute(insert_patient_query, patient_values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")
def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Appointment Time:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()

Removing appointment_time:

In [ ]:
import cv2
import pytesseract
import numpy as np
import spacy
import pandas as pd
import pdfplumber
from PIL import Image
from IPython.display import display, Markdown
import ipywidgets as widgets
import io
from ipyfilechooser import FileChooser
import re
import mysql.connector
from datetime import datetime

nlp = spacy.load("en_core_web_sm")

# Output widget for displaying extracted text and information
output_widget = widgets.Output()

# MySQL database connection details
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "ankit2061",
    "database": "ClaimDetails",
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        with output_widget:
            print("Error: Could not read the image. Please check the file format and path.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    kernel = np.ones((1, 1), np.uint8)
    processed_image = cv2.dilate(binary, kernel, iterations=1)
    processed_image = cv2.erode(processed_image, kernel, iterations=1)

    return processed_image

def extract_text_from_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    if preprocessed_image is None:
        return ""

    pil_image = Image.fromarray(preprocessed_image)
    try:
        text = pytesseract.image_to_string(pil_image)
    except pytesseract.TesseractError as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Tesseract Error: {e}")
        return ""
    return text.strip()

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"Error reading PDF: {e}")
        return ""
    return text.strip()

def extract_aadhaar_number(text):
    """
    Specialized function to extract Aadhaar numbers using multiple approaches
    """
    # Clean the text first - this can help with OCR errors
    cleaned_text = re.sub(r'\s+', ' ', text)
    
    # Method 1: Look for common Aadhaar number patterns (12 digits with or without separators)
    # This catches raw numbers that look like Aadhaar numbers
    aadhaar_patterns = [
        # Pattern with no separators - 12 consecutive digits
        r'(?<!\d)(\d{12})(?!\d)',
        # Pattern with space separators
        r'(\d{4}\s+\d{4}\s+\d{4})',
        # Pattern with dash separators
        r'(\d{4}-\d{4}-\d{4})',
        # Pattern with dot separators
        r'(\d{4}\.\d{4}\.\d{4})'
    ]
    
    for pattern in aadhaar_patterns:
        matches = re.findall(pattern, cleaned_text)
        if matches:
            # Clean up the found number (remove spaces, dashes)
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 2: Look for Aadhaar numbers with keywords
    # This catches Aadhaar numbers that are labeled
    keyword_patterns = [
        # Various ways "Aadhaar" might be written followed by a number
        r'(?:aadhar|aadhaar|adhar|aadha+r|आधार)(?:\s*(?:card|number|no|id|#|:|नंबर|संख्या))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        r'(?:uid|unique\s+id)(?:\s*(?:number|no|#))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        # Looking for "No:" or "Number:" followed by what could be an Aadhaar
        r'(?:no|number|id)?\s*[:\.\-]\s*((?:\d[\d\s\.\-]*){12})'
    ]
    
    for pattern in keyword_patterns:
        matches = re.findall(pattern, cleaned_text.lower())
        if matches:
            # Clean up the found number
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 3: More aggressive - find any 12-digit sequence that could be an Aadhaar number
    # Use with caution as it might pick up other 12-digit numbers
    digit_sequences = re.findall(r'(?<!\d)(\d[\d\s\.\-]*\d)(?!\d)', cleaned_text)
    for seq in digit_sequences:
        digits_only = re.sub(r'[^\d]', '', seq)
        if len(digits_only) == 12:
            return digits_only
            
    return None

def clean_extracted_field(text, field_type):
    """
    Cleans extracted text based on field type to remove common OCR artifacts
    and mislabeled content
    """
    # Convert to string in case we received another type
    text = str(text).strip()
    
    # Remove common label text that might be captured within the value
    unwanted_labels = [
        "Phone Number", "Contact", "Mobile", "Call",
        "Hospital Name", "Doctor", "Clinic", "MD", "Dr\\.",
        "Address", "Location", "Place", "Residence",
        "Insurance ID", "Policy Number", "Insurance",
        "Amount", "Total", "Fee", "Payment",
        "Disease", "Diagnosis", "Condition",
        "Medicines", "Medication", "Drugs", "Prescription"
    ]
    
    # For each unwanted label, try to remove it if it appears at the end
    for label in unwanted_labels:
        # Create pattern to match label at the end of the text (allowing for spaces)
        pattern = rf'\s*{re.escape(label)}$'
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # Remove common field separators
    text = re.sub(r'[:;|]$', '', text)
    
    # Clean up newlines and extra spaces
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Additional field-specific cleaning
    if field_type in ["Address"]:
        # Keep only relevant address information
        text = re.sub(r'\s*(?:Phone|Mobile|Contact|Email).*$', '', text, flags=re.IGNORECASE)
    
    elif field_type in ["Hospital Name"]:
        # Remove doctor references
        text = re.sub(r'\s*(?:Doctor|Dr\.|MD|Physician).*$', '', text, flags=re.IGNORECASE)
    
    elif field_type in ["Phone Number"]:
        # Keep only digits and basic formatting characters
        text = re.sub(r'[^\d+\-\s()]', '', text)
    
    return text.strip()

def extract_fields_with_boundaries(text):
    """
    Extract fields with improved boundary detection to prevent label bleed
    """
    extracted_info = []
    found_labels = set()
    
    # Dictionary of field patterns with better boundary detection
    field_patterns = {
        "Name": r'(?:Patient(?:\s*Name)?|Name|Patient)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Father|Gender|Blood|Aadhaar))',
        "Father's Name": r'(?:Father(?:[\'s]*\s*Name)?|Father)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Gender|Blood|Aadhaar))',
        "Gender": r'(?:Gender|Sex)[:;]?\s*(Male|Female|Other|M|F)(?=\n|$)',
        "Blood Group": r'(?:Blood(?:\s*Group)?)[:;]?\s*([ABO][+-]|AB[+-])(?=\n|$)',
        "Address": r'(?:Address|Location|Place|Residence)[:;]?\s*([\w\s,\.\-\/]+?)(?=\n|$|(?:Phone|Mobile|Contact|Email))',
        "Hospital Name": r'(?:Hospital(?:\s*Name)?|Clinic|Medical Center)[:;]?\s*([\w\s\.]+?)(?=\n|$|(?:Doctor|Dr|MD|Address))',
        "Insurance ID": r'(?:Insurance(?:\s*(?:ID|Number|No))?|Policy(?:\s*Number)?)[:;]?\s*([\w\d\-]+?)(?=\n|$)',
        "Phone Number": r'(?:Phone(?:\s*Number)?|Mobile|Contact|Cell)[:;]?\s*([\d\s\+\-\(\)]+?)(?=\n|$)',
        "Amount": r'(?:Amount|Total|Cost|Fee|Charges)[:;]?\s*([\d\.]+?)(?=\n|$|Rs|\$|₹)',
        "Disease Name": r'(?:Disease(?:\s*Name)?|Diagnosis|Condition|Ailment)[:;]?\s*([\w\s]+?)(?=\n|$|(?:Disease Details|Symptoms|Treatment))',
        "Disease Details": r'(?:Disease(?:\s*Details)?|Details|Diagnosis Details|Clinical Details|Symptoms)[:;]?\s*([\w\s,\.;\(\)\-\/]+?)(?=\n\n|\n(?:Medicines|Medications|Drugs)|$)',
        "Medicines": r'(?:Medicines|Medications|Drugs|Prescriptions|Medicine List)[:;]?\s*([\w\s,\.;\(\)\-\/]+?)(?=\n\n|\n(?:Bed|Ventilation|Amount|Charges)|$)',
        "Bed Type": r'(?:Bed(?:\s*Type)?)[:;]?\s*([\w\s]+?)(?=\n|$)',
        "Ventilation": r'(?:Ventilation|Ventilator|Oxygen)[:;]?\s*(Yes|No|Required|Not Required)(?=\n|$)',
        "Other Charges": r'(?:Other(?:\s*Charges)?|Additional(?:\s*Charges)?|Extra)[:;]?\s*([\d\.]+?)(?=\n|$|Rs|\$|₹)'
    }
    
    # 1. First pass: Extract Aadhaar number with dedicated function
    aadhaar = extract_aadhaar_number(text)
    if aadhaar:
        formatted_aadhaar = f"{aadhaar[:4]}-{aadhaar[4:8]}-{aadhaar[8:]}"
        extracted_info.append({"Text": formatted_aadhaar, "Label": "Aadhar Card"})
        found_labels.add("Aadhar Card")
    
    # 2. Second pass: Extract other fields with improved boundary detection
    for label, pattern in field_patterns.items():
        if label in found_labels:
            continue
            
        matches = re.search(pattern, text, re.IGNORECASE)
        if matches:
            extracted_text = matches.group(1).strip()
            # Clean the extracted text to remove potential label contamination
            cleaned_text = clean_extracted_field(extracted_text, label)
            
            # Only add if we have meaningful content
            if cleaned_text and len(cleaned_text) > 0:
                extracted_info.append({"Text": cleaned_text, "Label": label})
                found_labels.add(label)
    
    # 3. Third pass: Look for unlabeled numbers that might be specific fields
    if "Phone Number" not in found_labels:
        # Look for potential phone numbers (10-digit sequences)
        phone_matches = re.search(r'(?<!\d)(\d{10})(?!\d)', text)
        if phone_matches:
            extracted_info.append({"Text": phone_matches.group(1), "Label": "Phone Number"})
            found_labels.add("Phone Number")
    
    # Look for Appendicitis or other common conditions if disease name not found
    if "Disease Name" not in found_labels:
        common_diseases = ["appendicitis", "diabetes", "hypertension", "cancer", "fracture", "pneumonia"]
        for disease in common_diseases:
            if re.search(rf'\b{disease}\b', text, re.IGNORECASE):
                extracted_info.append({"Text": disease.capitalize(), "Label": "Disease Name"})
                found_labels.add("Disease Name")
                break
    
    return extracted_info

def process_text(text, keywords=[]):
    """
    Main processing function that combines extraction methods
    """
    # Get fields using improved boundary detection
    extracted_info = extract_fields_with_boundaries(text)
    
    # For backward compatibility, still use keyword-based extraction for any missing fields
    found_labels = {item["Label"] for item in extracted_info}
    
    for keyword in keywords:
        # Skip keywords for fields we already found
        label = keyword.replace(":", "").strip()
        if any(label in existing for existing in found_labels):
            continue
            
        # Simple keyword-based extraction as fallback
        pattern = re.compile(rf"{re.escape(keyword)}\s*([\w\s\d\.\-]+?)(?=\n|$)", re.IGNORECASE)
        match = pattern.search(text)
        if match:
            extracted_text = match.group(1).strip()
            cleaned_text = clean_extracted_field(extracted_text, label)
            
            if cleaned_text and len(cleaned_text) > 0:
                extracted_info.append({"Text": cleaned_text, "Label": label})
                found_labels.add(label)
    
    return extracted_info

# The extract_aadhaar_number function from previous solution should be included here
def extract_aadhaar_number(text):
    """
    Specialized function to extract Aadhaar numbers using multiple approaches
    """
    # Clean the text first - this can help with OCR errors
    cleaned_text = re.sub(r'\s+', ' ', text)
    
    # Method 1: Look for common Aadhaar number patterns (12 digits with or without separators)
    aadhaar_patterns = [
        # Pattern with no separators - 12 consecutive digits
        r'(?<!\d)(\d{12})(?!\d)',
        # Pattern with space separators
        r'(\d{4}\s+\d{4}\s+\d{4})',
        # Pattern with dash separators
        r'(\d{4}-\d{4}-\d{4})',
        # Pattern with dot separators
        r'(\d{4}\.\d{4}\.\d{4})'
    ]
    
    for pattern in aadhaar_patterns:
        matches = re.findall(pattern, cleaned_text)
        if matches:
            # Clean up the found number (remove spaces, dashes)
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 2: Look for Aadhaar numbers with keywords
    keyword_patterns = [
        # Various ways "Aadhaar" might be written followed by a number
        r'(?:aadhar|aadhaar|adhar|aadha+r|आधार)(?:\s*(?:card|number|no|id|#|:|नंबर|संख्या))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        r'(?:uid|unique\s+id)(?:\s*(?:number|no|#))?\s*[:\.\-]?\s*((?:\d[\d\s\.\-]*){12})',
        # Looking for "No:" or "Number:" followed by what could be an Aadhaar
        r'(?:no|number|id)?\s*[:\.\-]\s*((?:\d[\d\s\.\-]*){12})'
    ]
    
    for pattern in keyword_patterns:
        matches = re.findall(pattern, cleaned_text.lower())
        if matches:
            # Clean up the found number
            aadhaar = re.sub(r'[^\d]', '', matches[0])
            if len(aadhaar) == 12:
                return aadhaar
    
    # Method 3: More aggressive - find any 12-digit sequence that could be an Aadhaar number
    digit_sequences = re.findall(r'(?<!\d)(\d[\d\s\.\-]*\d)(?!\d)', cleaned_text)
    for seq in digit_sequences:
        digits_only = re.sub(r'[^\d]', '', seq)
        if len(digits_only) == 12:
            return digits_only
            
    return None
def save_to_database(data, insurance_id, file_path):
    try:
        cnx = mysql.connector.connect(**DB_CONFIG)
        cursor = cnx.cursor()

        # Insert document information into the 'patient_documents' table
        insert_doc_query = "INSERT INTO patient_documents (insurance_id, file_path) VALUES (%s, %s)"
        doc_values = (insurance_id, file_path)
        cursor.execute(insert_doc_query, doc_values)

        # Modified query to exclude the ventilation and appointment_time columns
        insert_patient_query = """
        INSERT INTO patient_details 
        (insurance_id, name, father_name, aadhar_card, gender, blood_group, 
        address, hospital_name, phone_number, amount, 
        disease_name, disease_details, medicines, bed_type, other_charges) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        name = next((item["Text"] for item in data if item["Label"] == "Name"), None)
        father_name = next((item["Text"] for item in data if item["Label"] == "Father's Name"), None)
        aadhar_card = next((item["Text"] for item in data if item["Label"] == "Aadhar Card"), None)
        gender = next((item["Text"] for item in data if item["Label"] == "Gender"), None)
        blood_group = next((item["Text"] for item in data if item["Label"] == "Blood Group"), None)
        address = next((item["Text"] for item in data if item["Label"] == "Address"), None)
        hospital_name = next((item["Text"] for item in data if item["Label"] == "Hospital Name"), None)
        phone_number = next((item["Text"] for item in data if item["Label"] == "Phone Number"), None)
        
        # Clean the amount value
        amount = next((item["Text"] for item in data if item["Label"] == "Amount"), None)
        if amount:
            amount = re.sub(r'[^\d.]', '', amount)
            if amount:
                try:
                    amount = float(amount)
                except ValueError:
                    amount = None
        
        disease_name = next((item["Text"] for item in data if item["Label"] == "Disease Name"), None)
        disease_details = next((item["Text"] for item in data if item["Label"] == "Disease Details"), None)
        medicines = next((item["Text"] for item in data if item["Label"] == "Medicines"), None)
        bed_type = next((item["Text"] for item in data if item["Label"] == "Bed Type"), None)
        
        # Clean other_charges
        other_charges = next((item["Text"] for item in data if item["Label"] == "Other Charges"), None)
        if other_charges:
            other_charges = re.sub(r'[^\d.]', '', other_charges)
            if other_charges:
                try:
                    other_charges = float(other_charges)
                except ValueError:
                    other_charges = None

        # Note: ventilation and appointment_time are removed from the values tuple
        patient_values = (insurance_id, name, father_name, aadhar_card, gender, blood_group, 
                          address, hospital_name, phone_number, amount, 
                          disease_name, disease_details, medicines, bed_type, other_charges)
        
        cursor.execute(insert_patient_query, patient_values)

        cnx.commit()
        cursor.close()
        cnx.close()
        with output_widget:
            print("Data saved to database successfully.")
    except mysql.connector.Error as err:
        with output_widget:
            print(f"Error saving to database: {err}")
def process_file(file_path):
    try:
        text = ""
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            text = extract_text_from_image(file_path)
        elif file_path.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)

        with output_widget:
            output_widget.clear_output()
            if not text.strip():
                print("Error: No text detected in the file.")
                return

            print("Full Extracted Text:", text)

            important_info = process_text(text, ["Name:", "Father's Name:", "Aadhar Card:", "Gender:", "Blood Group:", "Address:", "Hospital Name:", "Insurance ID:", "Phone Number:", "Amount:", "Disease Name:", "Disease Details:", "Medicines:", "Bed Type:", "Ventilation:", "Other Charges:"])
            if important_info:
                display(Markdown("### Important Extracted Information:"))
                df = pd.DataFrame(important_info)
                display(df)

                insurance_id_data = next((item["Text"] for item in important_info if item["Label"] == "Insurance ID"), None)

                if insurance_id_data is None:
                    print("Error: Insurance ID not found. Data not saved.")
                    return

                save_to_database(important_info, insurance_id_data, file_path)
            else:
                display(Markdown("**No important information found.**"))
    except Exception as e:
        with output_widget:
            output_widget.clear_output()
            print(f"An error occurred: {e}")

file_chooser = FileChooser()

def on_file_chosen(chooser):
    if chooser.selected:
        process_file(chooser.selected)

file_chooser.register_callback(on_file_chosen)

display(file_chooser, output_widget)

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Caskroom/miniconda/base/bin/tesseract'

FileChooser(path='/Users/ankittalukder/Documents/NeuraClaim', filename='', title='', show_hidden=False, select…

Output()